# Aula 3_4 - BERT

Nome:

## Instruções:


Criar um modelo de linguagem e medir a perplexidade utilizando o dataset do Machado de Assis, utilizando Embeddings gerados por um BERT pré-treinado e uma MLP.

- Deve-se implementar o próprio laço de treinamento. **Não usar frameworks de treinamento automático.**
- Utilizar o BertModel.from_pretrained e BertTokenizer.from_pretrained do Hugging Face para carregar o BERT pré-treinado.
- Não utilize outras classes da HuggingFace/Transformers alem da [BertModel](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel) e [BertTokenizer](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer).
- Trabalhe no espaço dos tokens/inteiros. Uma forma de fazer isso é tokenizar o dataset inteiro como pré-processamento.
- Experimente com aumentar o contexto e congelar ou não os parâmetros do BERT. Cuidado que o contexto é o maior fator para o peso computacional aqui.
- Sugerimos utilizar um BERT treinado em português como o BerTimbau: "neuralmind/bert-base-portuguese-cased".
- Inicialmente utilizar o hidden_state do token CLS. Podem experimentar com outras formas de usar o last_hidden_state.
- MLP deve utilizar o vocab_size do BERT na saída. Isso gera um grande desafio de manter o tamanho da MLP razoável.
- Deixe para usar GPU somente quando tudo estiver validado em parte pequena do dataset. Uma época pode demorar 30 minutos ou mais no dataset inteiro.

Opcional:
- Utilize bfloat16 para economizar memória da GPU.

In [ ]:
import random
import torch
import torch.nn.functional as F
import numpy as np

In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

## Dados

Vamos usar o mesmo dataset do Machado de Assis.



In [ ]:
!git clone https://github.com/ethelbeluzzi/projetomachado

Cloning into 'projetomachado'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 65 (delta 24), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 7.21 MiB | 3.64 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [ ]:
import os

DATA_PATH = os.path.join("projetomachado", "textonormalizado1000.txt")

# A príncipio, não estamos limpando as linhas
with open(DATA_PATH, "r") as data_file:
    lines = [line for line in data_file]

# É possível voltar a um texto monolítico juntando as linhas.
full_data = ' '.join(lines)
full_data[:1000]

'1\n MINISTÉRIO DA CULTURA\n Fundação Biblioteca Nacional\n Departamento Nacional do Livro\n A MÃO E A LUVA\n Machado de Assis\n I\n O fim da carta\n Mas que pretendes fazer agora?\n Morrer.\n Morrer? Que idéia! Deixate disso, Estêvão. Não se morre por tão pouco...\n Morrese. Quem não padece estas dores não as pode avaliar. O golpe foi profundo, e o\n meu coração é pusilânime; por mais aborrecível que pareça a idéia da morte, pior, muito pior do\n que ela, é a de viver. Ah! tu não sabes o que isto é?\n Sei: um namoro gorado...\n Luís!\n ... E se em cada caso de namoro gorado morresse um homem, tinha já diminuído muito o\n gênero humano, e Malthus perderia o latim. Anda, sobe.\n Estêvão meteu a mão nos cabelos com um gesto de angústia; Luís Alves sacudiu a cabeça\n e sorriu. Achavamse os dois no corredor da casa de Luís Alves, à rua da Constituição,  que\n então se chamava dos Ciganos;  então, isto é, em 1853, uma bagatela de vinte anos que lá vão,\n levando talvez consigo as ilusões do

In [ ]:
# Separar em treino e teste
limit = int(0.8*len(full_data))
train_data = full_data[:limit]
val_data = full_data[limit:]

# Não utilize o split val para nada a partir daqui, somente validar
len(train_data), len(val_data)

(15321482, 3830371)

## Tokenizer
-Testar com strings\
-Tokens especiais\
-Tokenize\
-Decode\
-Batch e padding


In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
tokenizer("Hello World").input_ids

[101, 7592, 2088, 102]

In [ ]:
tokenizer.decode([0, 101, 102, 103])

'[PAD] [CLS] [SEP] [MASK]'

In [ ]:
tokenizer("Hello World Orthogonalization")

{'input_ids': [101, 7592, 2088, 28721, 3989, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.tokenize("Hello World Orthogonalization")

['hello', 'world', 'orthogonal', '##ization']

In [ ]:
tokenizer.tokenize("Testando uma frase em português!")

['test',
 '##ando',
 'um',
 '##a',
 'fra',
 '##se',
 'em',
 'port',
 '##ug',
 '##ues',
 '!']

In [ ]:
tokenizer("Capitu e uma personagem")

{'input_ids': [101, 6178, 4183, 2226, 1041, 8529, 2050, 16115, 3351, 2213, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [ ]:
tokenizer("Capitu é uma personagem")

{'input_ids': [101, 2442, 6181, 253, 230, 2081, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Duas sentenças", "Outra sentença"], max_length=10, padding=True, truncation=True)

{'input_ids': [[101, 9551, 12385, 22281, 102], [101, 5802, 12385, 102, 0]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 0]]}

## Forward no modelo
Inspecionar modelo com torchinfo\
Visualizar saida do modelo

In [ ]:
!pip install torchinfo -q

In [ ]:
import torchinfo
from transformers import BertModel

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
tokenizer("Hello BERT, I am inspecting the model!", return_tensors='pt').input_ids

tensor([[  101, 15044, 22280,   241,  5054, 22321,   117,   290,  1052, 14949,
          5965,   446,  1621, 12066,   106,   102]])

In [ ]:
output = model().last_hidden_state
output[:, -1].shape

In [ ]:
# O que esta dentro da classe?
torchinfo.summary(model,
                  input_data=tokenizer("Hello BERT, I am inspecting the model!", return_tensors='pt').input_ids,
                  depth=3)

Layer (type:depth-idx)                                  Output Shape              Param #
BertModel                                               [1, 768]                  --
├─BertEmbeddings: 1-1                                   [1, 16, 768]              --
│    └─Embedding: 2-1                                   [1, 16, 768]              23,440,896
│    └─Embedding: 2-2                                   [1, 16, 768]              1,536
│    └─Embedding: 2-3                                   [1, 16, 768]              393,216
│    └─LayerNorm: 2-4                                   [1, 16, 768]              1,536
│    └─Dropout: 2-5                                     [1, 16, 768]              --
├─BertEncoder: 1-2                                      [1, 16, 768]              --
│    └─ModuleList: 2-6                                  --                        --
│    │    └─BertLayer: 3-1                              [1, 16, 768]              7,087,872
│    │    └─BertLayer: 3-2        

## Classe do dataset

Utilize o Tokenizer do Bert na classe Dataset.
Defina um tamanho de sequencia/contexto.

In [ ]:
context_size = 5 # 5 palavras de entrada. O target é a próxima palavra
"""TODO: Preparar o dataset"""

In [ ]:
"""TODO: implemente a classe do dataset"""

train_data = MyDataset(...)
val_data = MyDataset

95377

In [ ]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
sample = next(iter(train_loader))

## Model

In [ ]:
import torch.nn as nn

class LanguageModel(torch.nn.Module):
    """TODO: implementar o modelo de linguagem"""
    def __init__(self):
      raise NotImplementedError
      """TODO:
        Iniciliazar o BERT
        Inicializar MLP de predição de próxima palavra utilizando o hidden state do ultimo token no BERT
        Usar tamanho do vocabulário do BERT
        Sinta-se livre para também experimentar com outras formas de usar o last_hidden_state.
      """

    def forward(self):
      """TODO
      """
      raise NotImplementedError


In [ ]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
epochs = 10
lr = """TODO: experimento""""
criterion = """TODO CrossEntropy""""

optimizer = """TODO: AdamW ou outro""""

model.to(device)

"""TODO: Implemente o loop de treinamento. Em cada época, calcule e imprima a loss no dataset de validação""""

## Avaliação

In [ ]:
""" TODO: calcule a perplexidade final no dataset de validação """

## Exemplo de uso

In [ ]:
text = ""

def generate_text(model, vocab, text, max_length):
    """TODO: implemente a função para gerar texto até atingir o max_length"""

context = 5
max_length= 10
generate_text(text, max_length)